In [1]:
from pynq import Overlay
from pynq import MMIO
from pynq.lib.iic import AxiIIC
import cffi, time
import threading
import csv
import math

overlay = Overlay("/home/xilinx/pynq/overlays/no_cmd_ffnn.bit")
print(overlay.ip_dict.keys())

dict_keys(['top_ffnn_0', 'axi_iic_0', 'processing_system7_0'])


In [2]:
overlay.ip_dict['top_ffnn_0']


{'type': 'xilinx.com:hls:top_ffnn:1.0',
 'mem_id': 's_axi_control',
 'memtype': 'REGISTER',
 'gpio': {},
 'interrupts': {},
 'parameters': {'C_S_AXI_CONTROL_ADDR_WIDTH': '6',
  'C_S_AXI_CONTROL_DATA_WIDTH': '32',
  'Component_Name': 'design_1_top_ffnn_0_2',
  'II': '447',
  'clk_period': '10',
  'combinational': '0',
  'latency': '446',
  'machine': '64',
  'EDK_IPTYPE': 'PERIPHERAL',
  'C_S_AXI_CONTROL_BASEADDR': '0x40000000',
  'C_S_AXI_CONTROL_HIGHADDR': '0x4000FFFF',
  'ADDR_WIDTH': '6',
  'ARUSER_WIDTH': '0',
  'AWUSER_WIDTH': '0',
  'BUSER_WIDTH': '0',
  'CLK_DOMAIN': 'design_1_processing_system7_0_0_FCLK_CLK0',
  'DATA_WIDTH': '32',
  'FREQ_HZ': '100000000',
  'HAS_BRESP': '1',
  'HAS_BURST': '0',
  'HAS_CACHE': '0',
  'HAS_LOCK': '0',
  'HAS_PROT': '0',
  'HAS_QOS': '0',
  'HAS_REGION': '0',
  'HAS_RRESP': '1',
  'HAS_WSTRB': '1',
  'ID_WIDTH': '0',
  'INSERT_VIP': '0',
  'MAX_BURST_LENGTH': '1',
  'NUM_READ_OUTSTANDING': '1',
  'NUM_READ_THREADS': '1',
  'NUM_WRITE_OUTSTANDING

In [3]:
ADDR_BASE = 0x41600000
ADDR_RANGE = 0x10000
i2c_obj = MMIO(ADDR_BASE,ADDR_RANGE)
FFNN_ADDR_BASE = 0x40000000
FFNN_ADDR_RANGE = 0x10000
ffnn_obj = MMIO(FFNN_ADDR_BASE,FFNN_ADDR_RANGE)


In [4]:
i2c_obj.write(292,0)

In [5]:
iic_ip = overlay.ip_dict['axi_iic_0']
iic = AxiIIC(iic_ip)

ffi = cffi.FFI()

In [6]:
def read_(addr,reg,length=1):
    iic.send(addr,bytes([reg]),1,0)
    buf = ffi.new("unsigned char[]",length)
    iic.receive(addr, buf, length, 0)
    return [buf[i] for i in range(length)]

def write_(addr, reg, value):
    iic.send(addr, bytes([reg,value]), 2, 0)

def to_int16(low,high):
    val = (high << 8) | low
    if val & 0x8000:
        val -= 1 << 16
    return val

#하드웨어에 입력으로 넣기 위해 fixed16_8로 변환하는 함수
def float_to_fixed16_8(x):
    # 8 fractional bits → 256 곱함
    val = int(round(x * 256))  # 소수 반올림
    # 16비트 범위로 클리핑
    if val > 32767:
        val = 32767
    elif val < -32768:
        val = -32768
    return val


In [7]:
# BN0055 connection check
BN0055_addr = 0x28
chip_addr = 0x00
chip_id = read_(BN0055_addr, 0x00, 1)[0]
print("CHIP ID:", hex(chip_id)) #0xA0

CHIP ID: 0xa0


In [8]:
#NDOF mode on 0x0C

write_(BN0055_addr, 0x3D, 0X0C)
time.sleep(0.1)

In [9]:
#calibration check
cal = read_(BN0055_addr, 0x35, 1)[0]
sys = (cal >> 6) & 0x03
gyro = (cal >> 4) & 0x03
accel = (cal >> 2) & 0x03
mag = cal & 0x03

print(f"Cal : sys = {sys} Gyro = {gyro} accel = {accel} mag = {mag}")

Cal : sys = 0 Gyro = 3 accel = 3 mag = 3


In [10]:
# roboclaw encoder 
import serial.tools.list_ports

ports = serial.tools.list_ports.comports()

for port in ports:
    print(f"Device: {port.device} | Description: {port.description}")

import sys
sys.path.append("/home/xilinx/pynq/roboclaw")  # 실제 roboclaw_3.py가 있는 경로
from roboclaw_3 import Roboclaw

rc = Roboclaw("/dev/ttyACM0", 38400)  # 또는 ttyUSB0
rc.Open()

rc_address = 0x80

start_time_motor = time.time()

Device: /dev/ttyACM0 | Description: USB Roboclaw 2x30A


In [11]:
# ======================
# CSV 준비
# ======================
csv_file = "rover_data.csv"
with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow([
        "time","label", "delta_encL", "delta_encR",
        "heading", "cmdL", "cmdR"
    ])


In [12]:
def main_loop():
    # 전역 변수 선언: 모터 제어 및 센서 로깅에 필요한 모든 변수를 선언합니다.
    global pattern_idx, start_time_motor, cmdL, cmdR
    global prev_encL, prev_encR
    
    prev_encL, prev_encR = rc.ReadEncM1(rc_address)[1], rc.ReadEncM2(rc_address)[1]
    
    data = read_(BN0055_addr, 0x08, 44)
    initial_heading = to_int16(data[18], data[19]) / 16.0
    print(initial_heading)
    
    cmdL, cmdR = 15,15
    
    rc.ForwardM1(rc_address, cmdL)
    rc.ForwardM2(rc_address, cmdR)
    
    LOOP_PERIOD = 0.05
    initial_loops = 5  # 처음 4번은 그냥 입력만 사용
    loop_count = 0
    
    
    # 목표 주기 설정 (20Hz = 0.05초)
    LOOP_PERIOD = 0.05 
    CORRECTION_PERIOD = 0.15
    
    last_correction_time = time.time()
    
    while True:
        loop_start_time = time.time()
        loop_count += 1

        
        curr_time = loop_start_time
        
        # 엔코더 읽기 및 delta 계산
        encL = rc.ReadEncM1(rc_address)[1]
        encR = rc.ReadEncM2(rc_address)[1]
        
        # delta 계산 (이전 값 사용)
        delta_encL = encL - prev_encL
        delta_encR = encR - prev_encR
        
        # 다음 루프를 위한 이전 값 갱신
        prev_encL, prev_encR = encL, encR

        # IMU 읽기
        data = read_(BN0055_addr,0x08,44)
        heading = to_int16(data[18],data[19]) / 16.0
        
        heading_rad = math.radians(heading)

        sin_head = math.sin(heading_rad)
        cos_head = math.cos(heading_rad)
        
        
        input_data = [1,0,0,delta_encL/100,delta_encR/100, sin_head, cos_head]   
        fixed_input = [float_to_fixed16_8(v) for v in input_data]
        print(fixed_input)
        

        # write input
        # -----------
        INPUT_ADDR = 16

        num_input_words = (len(fixed_input) + 1) // 2
        for i in range(num_input_words):
            low = fixed_input[i*2] & 0xFFFF
            if i*2 + 1 < len(fixed_input):
                high = fixed_input[i*2 + 1] & 0xFFFF
            else:
                high = 0
            word = (high << 16) | low
            ffnn_obj.write(INPUT_ADDR + i*4, word)

        # -----------------------------
        # 4. ap_start = 1
        # CTRL register offset = 0x00
        # bit0 = ap_start
        # -----------------------------
        CTRL_ADDR = 0
        AP_START = 1

        ffnn_obj.write(CTRL_ADDR, AP_START)

        # -----------------------------
        # 5. ap_done 기다리기
        # CTRL bit1 = ap_done
        # -----------------------------
        AP_DONE_MASK = 1 << 1

        while True:
            ctrl_reg = ffnn_obj.read(CTRL_ADDR)
            if ctrl_reg & AP_DONE_MASK:
                break

        # -----------------------------
        # 6. 출력 읽기
        # -----------------------------
        OUTPUT_ADDR = 32

        output_fixed = []
        num_words = 1

        for i in range(num_words):   # num_words = 필요한 word 수 (출력값 / 2)
            word = ffnn_obj.read(OUTPUT_ADDR + i)
            low  = word & 0xFFFF            # lower 16bit
            high = (word >> 16) & 0xFFFF    # upper 16bit

            # signed 처리 (16비트 signed fixed)
            if low & 0x8000:
                low = -((~low & 0xFFFF) + 1)
            if high & 0x8000:
                high = -((~high & 0xFFFF) + 1)

            output_fixed.append(low)
            output_fixed.append(high)

        def fixed_to_float16_2(x):
            return float(x) / 16384.0               # 14 fractional bits

        output_float = [fixed_to_float16_2(v) for v in output_fixed]

        print("Raw Fixed Output:", output_fixed)
        print("Converted Float Output:", output_float)
        # ---------------------------
        # cmd값 변환
        # ---------------------------
        pred_cmdL = output_float[0] * 127
        pred_cmdR = output_float[1] * 127

        print(f"predicted cmdL={pred_cmdL:.2f}, cmdR={pred_cmdR:.2f}")
        
        if loop_count <= initial_loops:
            cmdL, cmdR = 15, 15
        else:
            cmdL = round(pred_cmdL)
            cmdR = round(pred_cmdR)
            

        
 


        
        rc.ForwardM1(rc_address, cmdL)
        rc.ForwardM2(rc_address, cmdR)
        
        label = "forward"

        # CSV 기록
        with open(csv_file, mode='a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([
                curr_time, label, delta_encL, delta_encR, heading, cmdL, cmdR
            ])
            
        # =======================# 3. 루프 시간 맞추기 (Loop Timing)# =======================
        loop_end_time = time.time()
        time_elapsed = loop_end_time - loop_start_time
        
        # 남은 시간만큼 대기하여 전체 루프 시간을 LOOP_PERIOD에 맞춥니다.
        sleep_time = LOOP_PERIOD - time_elapsed
        if sleep_time > 0:
            time.sleep(sleep_time)

In [13]:
main_loop()

355.8125
[256, 0, 0, 0, 0, -19, 255]
Raw Fixed Output: [6363, 5195]
Converted Float Output: [0.38836669921875, 0.31707763671875]
predicted cmdL=49.32, cmdR=40.27
[256, 0, 0, -36, -15, -19, 255]
Raw Fixed Output: [4761, 2795]
Converted Float Output: [0.29058837890625, 0.17059326171875]
predicted cmdL=36.90, cmdR=21.67
[256, 0, 0, -33, -23, -19, 255]
Raw Fixed Output: [4349, 2943]
Converted Float Output: [0.26544189453125, 0.17962646484375]
predicted cmdL=33.71, cmdR=22.81
[256, 0, 0, -44, -38, -19, 255]
Raw Fixed Output: [3097, 2261]
Converted Float Output: [0.18902587890625, 0.13800048828125]
predicted cmdL=24.01, cmdR=17.53
[256, 0, 0, -51, -49, -19, 255]
Raw Fixed Output: [2386, 1085]
Converted Float Output: [0.1456298828125, 0.06622314453125]
predicted cmdL=18.49, cmdR=8.41
[256, 0, 0, -54, -49, -20, 255]
Raw Fixed Output: [2386, 1292]
Converted Float Output: [0.1456298828125, 0.078857421875]
predicted cmdL=18.49, cmdR=10.01
[256, 0, 0, -61, -51, -20, 255]
Raw Fixed Output: [2261, 1

[256, 0, 0, -102, -108, -25, 255]
Raw Fixed Output: [2386, 3424]
Converted Float Output: [0.1456298828125, 0.208984375]
predicted cmdL=18.49, cmdR=26.54
[256, 0, 0, -102, -110, -25, 255]
Raw Fixed Output: [2386, 3597]
Converted Float Output: [0.1456298828125, 0.21954345703125]
predicted cmdL=18.49, cmdR=27.88
[256, 0, 0, -102, -108, -25, 255]
Raw Fixed Output: [2386, 3597]
Converted Float Output: [0.1456298828125, 0.21954345703125]
predicted cmdL=18.49, cmdR=27.88
[256, 0, 0, -102, -120, -25, 255]
Raw Fixed Output: [2386, 3597]
Converted Float Output: [0.1456298828125, 0.21954345703125]
predicted cmdL=18.49, cmdR=27.88
[256, 0, 0, -108, -105, -24, 255]
Raw Fixed Output: [2386, 3424]
Converted Float Output: [0.1456298828125, 0.208984375]
predicted cmdL=18.49, cmdR=26.54
[256, 0, 0, -105, -120, -24, 255]
Raw Fixed Output: [2386, 3597]
Converted Float Output: [0.1456298828125, 0.21954345703125]
predicted cmdL=18.49, cmdR=27.88
[256, 0, 0, -110, -110, -23, 255]
Raw Fixed Output: [2386, 359

[256, 0, 0, -97, -100, -17, 255]
Raw Fixed Output: [2386, 3097]
Converted Float Output: [0.1456298828125, 0.18902587890625]
predicted cmdL=18.49, cmdR=24.01
[256, 0, 0, -97, -95, -17, 255]
Raw Fixed Output: [2386, 3097]
Converted Float Output: [0.1456298828125, 0.18902587890625]
predicted cmdL=18.49, cmdR=24.01
[256, 0, 0, -97, -97, -16, 255]
Raw Fixed Output: [2386, 2943]
Converted Float Output: [0.1456298828125, 0.17962646484375]
predicted cmdL=18.49, cmdR=22.81
[256, 0, 0, -97, -95, -17, 255]
Raw Fixed Output: [2386, 2943]
Converted Float Output: [0.1456298828125, 0.17962646484375]
predicted cmdL=18.49, cmdR=22.81
[256, 0, 0, -100, -97, -17, 255]
Raw Fixed Output: [2386, 2943]
Converted Float Output: [0.1456298828125, 0.17962646484375]
predicted cmdL=18.49, cmdR=22.81
[256, 0, 0, -95, -90, -17, 255]
Raw Fixed Output: [2516, 2795]
Converted Float Output: [0.153564453125, 0.17059326171875]
predicted cmdL=19.50, cmdR=21.67
[256, 0, 0, -97, -97, -17, 255]
Raw Fixed Output: [2386, 2943]


[256, 0, 0, -95, -102, -15, 256]
Raw Fixed Output: [2386, 3097]
Converted Float Output: [0.1456298828125, 0.18902587890625]
predicted cmdL=18.49, cmdR=24.01
[256, 0, 0, -97, -97, -15, 256]
Raw Fixed Output: [2261, 3097]
Converted Float Output: [0.13800048828125, 0.18902587890625]
predicted cmdL=17.53, cmdR=24.01
[256, 0, 0, -100, -100, -15, 256]
Raw Fixed Output: [2386, 3097]
Converted Float Output: [0.1456298828125, 0.18902587890625]
predicted cmdL=18.49, cmdR=24.01
[256, 0, 0, -95, -97, -15, 256]
Raw Fixed Output: [2386, 2943]
Converted Float Output: [0.1456298828125, 0.17962646484375]
predicted cmdL=18.49, cmdR=22.81
[256, 0, 0, -97, -100, -15, 256]
Raw Fixed Output: [2386, 3097]
Converted Float Output: [0.1456298828125, 0.18902587890625]
predicted cmdL=18.49, cmdR=24.01
[256, 0, 0, -95, -92, -15, 256]
Raw Fixed Output: [2386, 2943]
Converted Float Output: [0.1456298828125, 0.17962646484375]
predicted cmdL=18.49, cmdR=22.81
[256, 0, 0, -95, -100, -15, 256]
Raw Fixed Output: [2386, 2

[256, 0, 0, -44, -36, -14, 256]
Raw Fixed Output: [2261, 1534]
Converted Float Output: [0.13800048828125, 0.0936279296875]
predicted cmdL=17.53, cmdR=11.89
[256, 0, 0, -36, -33, -14, 256]
Raw Fixed Output: [2142, 1292]
Converted Float Output: [0.1307373046875, 0.078857421875]
predicted cmdL=16.60, cmdR=10.01
[256, 0, 0, -69, -36, -15, 256]
Raw Fixed Output: [2261, 1150]
Converted Float Output: [0.13800048828125, 0.0701904296875]
predicted cmdL=17.53, cmdR=8.91
[256, 0, 0, -59, -31, -16, 256]
Raw Fixed Output: [2386, 1023]
Converted Float Output: [0.1456298828125, 0.06243896484375]
predicted cmdL=18.49, cmdR=7.93
[256, 0, 0, -59, -36, -16, 255]
Raw Fixed Output: [2516, 1023]
Converted Float Output: [0.153564453125, 0.06243896484375]
predicted cmdL=19.50, cmdR=7.93
[256, 0, 0, -64, -41, -17, 255]
Raw Fixed Output: [2516, 1023]
Converted Float Output: [0.153564453125, 0.06243896484375]
predicted cmdL=19.50, cmdR=7.93
[256, 0, 0, -67, -49, -18, 255]
Raw Fixed Output: [2653, 1085]
Converted

...

[256, 0, 0, -100, -110, -12, 256]
Raw Fixed Output: [2386, 3097]
Converted Float Output: [0.1456298828125, 0.18902587890625]
predicted cmdL=18.49, cmdR=24.01
[256, 0, 0, -97, -90, -12, 256]
Raw Fixed Output: [2386, 3097]
Converted Float Output: [0.1456298828125, 0.18902587890625]
predicted cmdL=18.49, cmdR=24.01
[256, 0, 0, -100, -97, -12, 256]
Raw Fixed Output: [2386, 2943]
Converted Float Output: [0.1456298828125, 0.17962646484375]
predicted cmdL=18.49, cmdR=22.81
[256, 0, 0, -97, -97, -12, 256]
Raw Fixed Output: [2386, 2795]
Converted Float Output: [0.1456298828125, 0.17059326171875]
predicted cmdL=18.49, cmdR=21.67
[256, 0, 0, -100, -95, -12, 256]
Raw Fixed Output: [2516, 2795]
Converted Float Output: [0.153564453125, 0.17059326171875]
predicted cmdL=19.50, cmdR=21.67
[256, 0, 0, -108, -102, -12, 256]
Raw Fixed Output: [2516, 2653]
Converted Float Output: [0.153564453125, 0.16192626953125]
predicted cmdL=19.50, cmdR=20.56
[256, 0, 0, -102, -92, -12, 256]
Raw Fixed Output: [2516, 26

[256, 0, 0, -82, -67, -20, 255]
Raw Fixed Output: [2653, 1919]
Converted Float Output: [0.16192626953125, 0.11712646484375]
predicted cmdL=20.56, cmdR=14.88
[256, 0, 0, -82, -77, -20, 255]
Raw Fixed Output: [2653, 2028]
Converted Float Output: [0.16192626953125, 0.123779296875]
predicted cmdL=20.56, cmdR=15.72
[256, 0, 0, -82, -77, -20, 255]
Raw Fixed Output: [2516, 2142]
Converted Float Output: [0.153564453125, 0.1307373046875]
predicted cmdL=19.50, cmdR=16.60
[256, 0, 0, -82, -82, -20, 255]
Raw Fixed Output: [2516, 2261]
Converted Float Output: [0.153564453125, 0.13800048828125]
predicted cmdL=19.50, cmdR=17.53
[256, 0, 0, -84, -84, -20, 255]
Raw Fixed Output: [2516, 2386]
Converted Float Output: [0.153564453125, 0.1456298828125]
predicted cmdL=19.50, cmdR=18.49
[256, 0, 0, -87, -87, -20, 255]
Raw Fixed Output: [2386, 2653]
Converted Float Output: [0.1456298828125, 0.16192626953125]
predicted cmdL=18.49, cmdR=20.56
[256, 0, 0, -87, -87, -20, 255]
Raw Fixed Output: [2386, 2795]
Conver

[256, 0, 0, -72, -64, -28, 254]
Raw Fixed Output: [2386, 1919]
Converted Float Output: [0.1456298828125, 0.11712646484375]
predicted cmdL=18.49, cmdR=14.88
[256, 0, 0, -72, -67, -28, 254]
Raw Fixed Output: [2386, 2028]
Converted Float Output: [0.1456298828125, 0.123779296875]
predicted cmdL=18.49, cmdR=15.72
[256, 0, 0, -69, -69, -28, 254]
Raw Fixed Output: [2386, 2261]
Converted Float Output: [0.1456298828125, 0.13800048828125]
predicted cmdL=18.49, cmdR=17.53
[256, 0, 0, -79, -79, -29, 254]
Raw Fixed Output: [2386, 2386]
Converted Float Output: [0.1456298828125, 0.1456298828125]
predicted cmdL=18.49, cmdR=18.49
[256, 0, 0, -77, -79, -29, 254]
Raw Fixed Output: [2386, 2653]
Converted Float Output: [0.1456298828125, 0.16192626953125]
predicted cmdL=18.49, cmdR=20.56
[256, 0, 0, -82, -87, -29, 254]
Raw Fixed Output: [2261, 2943]
Converted Float Output: [0.13800048828125, 0.17962646484375]
predicted cmdL=17.53, cmdR=22.81
[256, 0, 0, -87, -95, -29, 254]
Raw Fixed Output: [2261, 3258]
Con

[256, 0, 0, -2209, -2396, -9, 256]
Raw Fixed Output: [530, 1085]
Converted Float Output: [0.0323486328125, 0.06622314453125]
predicted cmdL=4.11, cmdR=8.41
[256, 0, 0, -92, -105, -9, 256]
Raw Fixed Output: [857, 1717]
Converted Float Output: [0.05230712890625, 0.10479736328125]
predicted cmdL=6.64, cmdR=13.31
[256, 0, 0, -56, -56, -9, 256]
Raw Fixed Output: [857, 1717]
Converted Float Output: [0.05230712890625, 0.10479736328125]
predicted cmdL=6.64, cmdR=13.31
[256, 0, 0, -61, -56, -9, 256]
Raw Fixed Output: [1023, 1368]
Converted Float Output: [0.06243896484375, 0.08349609375]
predicted cmdL=7.93, cmdR=10.60
[256, 0, 0, -44, -51, -9, 256]
Raw Fixed Output: [530, 1085]
Converted Float Output: [0.0323486328125, 0.06622314453125]
predicted cmdL=4.11, cmdR=8.41
[256, 0, 0, -31, -31, -9, 256]
Raw Fixed Output: [2142, 1534]
Converted Float Output: [0.1307373046875, 0.0936279296875]
predicted cmdL=16.60, cmdR=11.89
[256, 0, 0, -36, -26, -9, 256]
Raw Fixed Output: [2028, 1219]
Converted Float


KeyboardInterrupt



In [14]:
rc.ForwardM1(rc_address, 0)
rc.ForwardM2(rc_address, 0)

True

In [ ]:
        if cmdL > 25:
            cmdL = 25
        elif cmdL < 5.0:
            cmdL = 5
        else:
            cmdL = cmdL

        if cmdR > 25:
            cmdR = 25
        elif cmdR < 5.0:
            cmdR = 5
        else:
            cmdR = cmdR